# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!


  1. Use RAGAS to Generate Synthetic Data
  2. Load them into a LangSmith Dataset
  3. Evaluate our RAG chain against the synthetic test data
  4. Make changes to our pipeline
  5. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [34]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/allantan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/allantan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [1]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [2]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [6]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [7]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [8]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 7922ab85-b00b-49f7-909c-abd12c9aa4d9 does not have a summary. Skipping filtering.
Node 0bd37c88-c5be-4af7-a87e-5bf4e35c8c22 does not have a summary. Skipping filtering.
Node 274fa1fe-5ffb-4446-9a82-7c4236058eef does not have a summary. Skipping filtering.
Node 6140f190-bd41-421b-8036-f58a72674817 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 138)

We can save and load our knowledge graphs as follows.

In [9]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 138)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [11]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

Single-Hop Query (Specific) – Creates fact-based questions that can be answered from a single piece of information.

Multi-Hop Query (Specific) – Generates questions that require putting together several pieces of information, but the question is focused on specific facts or details.

Multi-Hop Query (Abstract) – Also needs multiple pieces of info, but the questions are more general or conceptual.
</span>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [12]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What does PIA S. CAYETANO do about AI regulati...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,PIA S. CAYETANO introduced an act regulating t...,single_hop_specifc_query_synthesizer
1,What is the concern about Artificial Superinte...,[AI presents enormous opportunities for the Ph...,The concerns regarding Artificial Superintelli...,single_hop_specifc_query_synthesizer
2,What does Article XIV refer to in the context ...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,Article XIV is mentioned as part of the Philip...,single_hop_specifc_query_synthesizer
3,How does the regulation of AI address the deve...,"[1 \na) Promote innovation, technological adva...","The regulation of AI, as outlined in the conte...",single_hop_specifc_query_synthesizer
4,What is the meaning of Artificial Superintelli...,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Artificial Superintelligence (ASI) refers to h...,single_hop_specifc_query_synthesizer
5,how AI superintelligence (ASI) might be regula...,[<1-hop>\n\nAI presents enormous opportunities...,"The context discusses the risks of AI, includi...",multi_hop_abstract_query_synthesizer
6,How do the NAIC and AI Ethics Review Board enf...,[<1-hop>\n\n1 \niii) Mandatory compliance trai...,The NAIC and AI Ethics Review Board enforce re...,multi_hop_abstract_query_synthesizer
7,how restrictions on AI use during elections or...,[<1-hop>\n\n1 \niii) Mandatory compliance trai...,The context shows that there are restrictions ...,multi_hop_abstract_query_synthesizer
8,How does the Artificial Intelligence Regulatio...,"[<1-hop>\n\n1 \na) Promote innovation, technol...",The Artificial Intelligence Regulation Act (AI...,multi_hop_specific_query_synthesizer
9,How does the potential rise of Artificial Supe...,[<1-hop>\n\nAI presents enormous opportunities...,The context highlights concerns about the poss...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node fa5da5f1-5ce8-4374-8243-acdc8738722c does not have a summary. Skipping filtering.
Node 820991fe-4516-4b19-8adf-4020cc42df20 does not have a summary. Skipping filtering.
Node 97bb5805-f5be-4825-bc78-6f6afeac48f2 does not have a summary. Skipping filtering.
Node d541fd7b-b130-423e-8cf8-b692e6f84c1d does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [14]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of the Senate in regulating a...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The Senate introduced an act regulating the de...,single_hop_specifc_query_synthesizer
1,What role does Georgetown University play in a...,[AI presents enormous opportunities for the Ph...,The context references Georgetown University i...,single_hop_specifc_query_synthesizer
2,What does the Philippine Constitution say abou...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The provided context does not specify the exac...,single_hop_specifc_query_synthesizer
3,What is the correct spelling of the term that ...,"[1 \na) Promote innovation, technological adva...",The term is spelled as Artificial General Inte...,single_hop_specifc_query_synthesizer
4,"How do the responsibilities of AI developers, ...",[<1-hop>\n\n1 \nSec. 15. AI Ethics Review Boar...,"The responsibilities of AI developers, deploye...",multi_hop_abstract_query_synthesizer
5,how AI and sustainable innovation hubs help in...,[<1-hop>\n\n1 \niii) \nProof of employer engag...,The context shows that AI and sustainable inno...,multi_hop_abstract_query_synthesizer
6,"what responsibilities AI developers, deployers...",[<1-hop>\n\n1 \nSec. 15. AI Ethics Review Boar...,"AI developers, deployers, and operators are re...",multi_hop_abstract_query_synthesizer
7,Considering the strict penalties outlined for ...,[<1-hop>\n\n1 \niii) Mandatory compliance trai...,The misrepresentation of AI-generated content ...,multi_hop_abstract_query_synthesizer
8,How does the DOST's establishment of the Natio...,[<1-hop>\n\n1 \nground its responses in verifi...,The context indicates that the Department of S...,multi_hop_specific_query_synthesizer
9,What is the role of DOST in AI regulation and ...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,The DOST is involved in AI regulation through ...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [15]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [16]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [17]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [20]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [21]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [22]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [24]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [25]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to create or disseminate disinformation is a fine of One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000), or imprisonment of three (3) years to ten (10) years, or both, at the discretion of the court. Additional penalties may apply if such acts are committed during election periods, public emergencies, or in violation of constitutional rights.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [26]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [27]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

qa_evaluator: evaluates the correctness of the model's answer compared to a reference/ground truth answer.

labeled_helpfulness_evaluator: evaluates how helpful the answering is by comparing it to the correct answer, focusing on usefulness to the user/s.

empathy_evaluator: evaluates whether the response shows empathy and makes the user feel heard and understood.
</div>

## LangSmith Evaluation

In [28]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'cold-ship-56' at:
https://smith.langchain.com/o/1416e1a2-8bd8-4452-a1d2-3cea46dfc419/datasets/d08bbe12-8d6d-4124-acf8-84b879080d4c/compare?selectedSessions=2e039fef-5ab7-419d-8a95-9c7634686fa3




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the NAIC's regulation of AI in employ...,"Based on the provided context, the NAIC (Natio...",None,The NAIC is tasked with regulating AI use in e...,1,1,0,7.761246,05f10658-641c-43fd-a3fe-62f083a1fce6,e9216590-5a37-488b-8ef6-b16c47b90365
1,How does the NAIC's jurisdiction over AI devel...,Based on the provided context:\n\n- The NAIC (...,None,The NAIC has exclusive jurisdiction over all m...,0,0,0,5.668202,fa12f630-d94a-4305-8c18-a0024fd4c00c,8dc77caf-dcf3-4e32-b0ee-009eea6919e4
2,What is the role of DOST in AI regulation and ...,"Based on the provided context, the Department ...",None,The DOST is involved in AI regulation through ...,1,1,0,6.139733,b6d02883-5d6e-4a7a-93de-b792144db092,292fc71d-5539-4cf2-9bcc-4e052bb28be7
3,How does the DOST's establishment of the Natio...,"Based on the provided context, the Department ...",None,The context indicates that the Department of S...,0,0,0,5.736096,1adcbccc-fbae-405f-a594-ddced49ed040,1a2290b7-a0ac-413c-b251-48d7d3401ffd
4,Considering the strict penalties outlined for ...,"Based on the provided context, the misrepresen...",None,The misrepresentation of AI-generated content ...,1,1,0,4.256857,aa0daf5e-e71e-4f33-be25-c0d23ff9b727,3c500e00-23fb-4aa3-867c-b8fcd9e350c1
5,"what responsibilities AI developers, deployers...","Based on the provided context, the responsibil...",None,"AI developers, deployers, and operators are re...",1,1,0,4.479522,8f3be026-7ac8-443d-a008-928b7333dcde,cb6c5e7c-6f24-4c2c-9ffb-b9d343a82860
6,how AI and sustainable innovation hubs help in...,Based on the provided context:\n\n**AI and Sus...,None,The context shows that AI and sustainable inno...,1,1,1,7.148225,1d503d11-6917-411d-aab3-f2e3193f22f8,572792a0-a61a-40ee-aa43-5e77e96d5e49
7,"How do the responsibilities of AI developers, ...","Based on the provided context, the responsibil...",None,"The responsibilities of AI developers, deploye...",1,1,0,4.739570,6b9cc154-aff2-462d-9cc9-0e6398d0472e,bb053e37-0f04-46e0-818e-160c061e85c3
8,What is the correct spelling of the term that ...,"The correct spelling of the term is ""Artificia...",None,The term is spelled as Artificial General Inte...,0,0,0,1.477606,cbeed7dc-f78d-4e3a-8bd7-6f4565428b93,e793922a-46e7-4fa2-8b5e-9f44b951e89d
9,What does the Philippine Constitution say abou...,I don't know.,None,The provided context does not specify the exac...,0,0,0,1.378847,a989c2d8-28df-4152-a38f-3c64b8fa3695,0d4c3965-ea0c-42b6-b69a-c55f0c351faa


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [29]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [30]:
rag_documents = docs

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:
Large chunks can contain more context in each piece, while smaller chunks are more focused. Modifying the chunk size changes how much and what kind of context is available to the model, which directly impacts answer quality.
</div>

In [32]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :
The embedding model turns text into numbers (vectors) so the system can find and compare relevant information. When we use a better embedding model (e.g., text-embedding-3-large), it creates more accurate and meaningful representations of the text. Modifying the embedding model changes how well the system understands and retrieves relevant information, which directly impacts answer quality.
</div>

In [33]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG 2"
)

In [34]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [35]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [36]:
empathy_rag_chain.invoke({"question" : "Why is the Philippines AI Bill important?"})

"Thank you for your thoughtful question about the importance of the Philippines AI Bill. Based on the context, this bill is crucial because it aims to create a national framework that ensures the development and use of artificial intelligence in a way that is safe, ethical, transparent, and under meaningful human oversight. It seeks to balance encouraging technological innovation with protecting the rights and welfare of every citizen.\n\nThe bill recognizes that AI is transforming industries and society, and as the Philippines advances in its digital journey, it’s important to have responsible and lawful guidelines to prevent misuse of AI, such as crime, rights abuse, or unintended harm. Moreover, it envisions AI as a tool to support Filipino ingenuity and address national development challenges.\n\nOverall, the bill reflects a genuine commitment to fostering innovation while safeguarding people and promoting a secure and inclusive digital future. I hope this helps clarify why the bil

Finally, we can evaluate the new chain on the same test set!

In [37]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'memorable-speed-79' at:
https://smith.langchain.com/o/1416e1a2-8bd8-4452-a1d2-3cea46dfc419/datasets/d08bbe12-8d6d-4124-acf8-84b879080d4c/compare?selectedSessions=14f61d62-6253-4458-97a0-83ae23a79921




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the NAIC's regulation of AI in employ...,Thank you for your thoughtful question. From t...,None,The NAIC is tasked with regulating AI use in e...,1,1,1,9.664713,05f10658-641c-43fd-a3fe-62f083a1fce6,8aaa00c6-1ca5-4b3a-9843-4e192a4199d8
1,How does the NAIC's jurisdiction over AI devel...,Thank you for your thoughtful question. It’s c...,None,The NAIC has exclusive jurisdiction over all m...,1,1,1,5.615439,fa12f630-d94a-4305-8c18-a0024fd4c00c,2743dd23-62cf-4362-8e64-5c8c83fec373
2,What is the role of DOST in AI regulation and ...,Thank you for your thoughtful question. Based ...,None,The DOST is involved in AI regulation through ...,1,0,1,3.735421,b6d02883-5d6e-4a7a-93de-b792144db092,b9017d68-fa70-4415-a930-a46b6d877aa2
3,How does the DOST's establishment of the Natio...,Thank you for your thoughtful question. It’s c...,None,The context indicates that the Department of S...,1,1,1,6.134700,1adcbccc-fbae-405f-a594-ddced49ed040,32349c24-a0c3-4a79-8213-cd064bfed31f
4,Considering the strict penalties outlined for ...,Thank you for your thoughtful and important qu...,None,The misrepresentation of AI-generated content ...,1,1,1,7.252160,aa0daf5e-e71e-4f33-be25-c0d23ff9b727,868f6e16-8ca8-4750-bead-c607de629337
5,"what responsibilities AI developers, deployers...",Thank you for your thoughtful question. Based ...,None,"AI developers, deployers, and operators are re...",1,1,1,5.198464,8f3be026-7ac8-443d-a008-928b7333dcde,72efad54-c055-4c9b-a810-94c904b66909
6,how AI and sustainable innovation hubs help in...,Thank you for your thoughtful question. It sou...,None,The context shows that AI and sustainable inno...,1,1,1,6.572762,1d503d11-6917-411d-aab3-f2e3193f22f8,c84f3e31-e5df-4a63-ae32-7b306696af77
7,"How do the responsibilities of AI developers, ...",Thank you for your thoughtful question. Based ...,None,"The responsibilities of AI developers, deploye...",1,0,1,7.197914,6b9cc154-aff2-462d-9cc9-0e6398d0472e,c76ed9b7-f0e8-4b27-a827-9b29297ef530
8,What is the correct spelling of the term that ...,Thank you for your thoughtful question. Based ...,None,The term is spelled as Artificial General Inte...,1,1,1,3.335651,cbeed7dc-f78d-4e3a-8bd7-6f4565428b93,88b97c24-8cb5-4ae1-ace9-b4ab81e065c5
9,What does the Philippine Constitution say abou...,Thank you for your thoughtful question. Based ...,None,The provided context does not specify the exac...,0,0,1,3.523630,a989c2d8-28df-4152-a38f-3c64b8fa3695,f2ce411d-113f-4737-8dd1-7b6040253f02


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Explain why you believe certain metrics changed in certain ways, and provide a screenshot of the difference between the two chains.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:
The metrics changed because the new chain used better embeddings, bigger chunks of context, and a new prompt that made the answers more empathetic. This helped increase in correctness, helpfulness, and especially empathy. However, the downside was a small increase in latency because the model had to process more information.

First chain:
<img src="./images/first_chain.png" alt="Metric Comparison - First Chain" />

Second chain:
<img src="./images/second_chain.png" alt="Metric Comparison - Second Chain" />
</div>